# Customer No-Show Prediction with XGBoost
_**Using Gradient Boosted Trees to predict Medical Appointment No-Shows**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)


---

## Background

_This notebook has been adapted from an [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and source or presigned url to the S3 bucket containg the data that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [ ]:
import boto3
import re
import time
import sagemaker
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker import get_execution_role

In [ ]:
# Set session 
sm = sagemaker.Session()

# Set default bucket for model artifacts and data channels
bucket = sm.default_bucket()

# SageMaker Role
role = get_execution_role()

---
## Data

The Kaggle dataset comprised 110k appointments records from public healthcare institutions in a Brazilian city. The appointments occurred across a 6-week period in 2016.

110.527 medical appointments its 14 associated variables (characteristics)

## Data Dictionary

01 - PatientId
Identification of a patient

02 - AppointmentID
Identification of each appointment

03 - Gender
Male or Female . Female is the greater proportion, woman takes way more care of they health in comparison to man.

04 - DataMarcacaoConsulta
The day of the actuall appointment, when they have to visit the doctor.

05 - DataAgendamento
The day someone called or registered the appointment, this is before appointment of course.

06 - Age
How old is the patient.

07 - Neighbourhood
Where the appointment takes place.

08 - Scholarship
True of False . Observation, this is a broad topic, consider reading this article https://en.wikipedia.org/wiki/Bolsa_Fam%C3%ADlia

09 - Hipertension
True or False

10 - Diabetes
True or False

11 - Alcoholism
True or False

12 - Handcap
True or False

13 - SMS_received
1 or more messages sent to the patient.

14 - No_show
 True or False.


## Exploratory data analysis & feature engineering 

In [ ]:
# Read the data into a Pandas Data Frame and inspect the data
df = pd.read_csv('no-show-data.csv')
df.head(n=10)

In [ ]:
# Correct misspelling of column names
df.rename(columns = {'ApointmentData':'AppointmentData',
                         'Alcoolism': 'Alchoholism',
                         'Hipertension': 'Hypertension',
                         'Handcap': 'Handicap',
                         'No-show': 'No_show'}, inplace = True)

print(df.columns)

In [ ]:
pd.set_option('display.max_columns', 100)
df.head(n=10)

In [ ]:
# Check for any missing data
df.info()

In [ ]:
# Frequency tables for some of the categorical feature(s)
# for column in df.select_dtypes(include=['object']).columns:
for column in df[[ 'Scholarship','Gender','No_show', 'Diabetes', 'Hypertension','Alcoholism','Handicap']]:       
    display(pd.crosstab(index=df[column], columns='% observations', normalize='columns'))

In [ ]:
# Histograms for each feature
display(df.describe())
%matplotlib inline
hist = df.hist(bins=40, sharey=True, figsize=(20, 8))


In [ ]:
display(df.corr())
pd.plotting.scatter_matrix(df, figsize=(15, 15))
plt.show()

Relationship on 'Scholarship', 'Hypertension','Diabetes', 'Alcoholism', 'Handicap', 'SMS_received' with No_show make proportions for each elements to find the relationships


In [ ]:
# Relationship on 'Scholarship', 'Hypertension','Diabetes', 'Alcoholism', 'Handicap', 'SMS_received' with No_show - Plot proportions for each elements to find the relationships
df_new = df.groupby('No_show')['Scholarship', 'Hypertension',\
                       'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received'].sum()
noshow_6r = df_new.query("No_show == 'Yes'")
noshow_total = df['No_show'].value_counts()[1]
prop_6r = noshow_6r / noshow_total

In [ ]:
sns.set_style('darkgrid')
prop_6r.plot(kind='bar',figsize=(16,8),\
            title='Relationship on 6 factors with No-Show')

## Data Wrangling

In [ ]:
# Cast to int64
df['PatientId'] = df['PatientId'].astype('int64')

In [ ]:
# Make PatientID the index column and No_show the first column in the dataframe (SageMaker's XGBoost expects the target variable to be in the first column when using csv)
df.set_index('PatientId', inplace = True)
df = df[ ['No_show'] + [ col for col in df.columns if col != 'No_show' ] ]

In [ ]:
# Check changes applied
df.head(n=5)

From the histograms abobe we can see immediately that 'Age" has negative values and needs to be further evaluated


In [ ]:
df['Age'].describe()

In [ ]:
# Distribution of 'Age'
plt.figure();
age_hist = df['Age'].plot.hist(bins=10)
age_hist.set_xlabel("Age")
age_hist.set_ylabel("Patients")
age_hist.set_title('Distribution of Age')

In [ ]:
# Only keep records of patients where they are between 0-99 in age to remove outliers.
df = df[(df.Age >= 0) & (df.Age <= 99)]

min_age = df['Age'].min()
max_age = df['Age'].max()
print ("Age now spans values from: {} to {}.".format(min_age, max_age))

In [ ]:
# Next let's look at the relationship between each of the features and our target variable
for column in df.select_dtypes(include=['object']).columns:
    if column != 'No_show':
        display(pd.crosstab(index=df[column], columns=df['No_show'], normalize='columns'))

for column in df.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = df[[column, 'No_show']].hist(by='No_show', bins=30)
    plt.show()

Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format.  For this example, we'll stick with CSV.  It should:
- Have the predictor variable in the first column
- Not have a header row

But first, let's convert our categorical features into numeric features.

In [ ]:
### This feature is suppose to be true/false 
df['Handicap'].replace([2,3,4],1, inplace = True)

In [ ]:
# Replace 'M' and 'F' with 1 and 0 for 'Gender' and 'Yes' and 'No'
# with 1 and 0 for 'No_show'
df['Gender'] = df['Gender'].map({'M':1, 
                                 'F':0}
                               )
df['No_show'] = df['No_show'].map({'Yes':1, 
                                   'No':0}
                                 )

In [ ]:
# Transforming ScheduledDay and AppointmentDay into datetime objects and stripping hours, minutes and seconds.
dt_scheduledDay =  pd.to_datetime(df.ScheduledDay).dt.date
dt_appointmentDay = pd.to_datetime(df.AppointmentDay).dt.date

# Storing "delta_days" to df as a new feature
df['Days_delta'] = (dt_appointmentDay - dt_scheduledDay).dt.days
df.head()

In [ ]:
# Changing AppointmentDay to a datetime pandas object to create a new dayofweek engineered feature
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df['No_show_weekday'] = df['AppointmentDay'].dt.dayofweek
#df['No_show_weekday'] = df[['No_show', 'No_show_weekday']].groupby('No_show_weekday').mean()
#df['No_show_weekday'].sample(5)
sns.barplot(y='No_show', x='No_show_weekday', data=df)

In [ ]:
# days_delta contains impossible values such as -6 and -1 
# which look like mistakes/outliers but require further investigation.

days_hist = df['Days_delta'].plot.hist(bins=8)
days_hist.set_xlabel("Days delta")
days_hist.set_xticks(range(0, 180, 10))
days_hist.set_ylabel("Patients")
days_hist.set_title('Distribution of Days delta')

In [ ]:
# days_delta < 0 and > 90 are not on the histogram which indicates that they don't belong to a patient/few patients and therefore should be removed.
df = df[(df.Days_delta >= 0) & (df.Days_delta <= 70)]

In [ ]:
# Creating vars to hold categorical features for one hot encoding
cat_features_for_encoding = ['Handicap', 'Neighbourhood']

# Creating var for all numerical features in case we want to deep dive to identify
# which age and what days delta are the most important in predicting "No_show" outcome
num_features_for_encoding = ['Age', 'Days_delta']

In [ ]:
# One-Hot encoding on categorical columns to prepare dataset for machine learning modelling.
encoded_df = pd.get_dummies(df, columns=cat_features_for_encoding)
print ("New encoded dataframe has {} rows and {} features.".format(encoded_df.shape[0], encoded_df.shape[1]))

# Increasing the max column shown for this cell to 100
pd.set_option("max_columns", 100)

In [ ]:
# Dropping columns that we have encoded
features = encoded_df.drop(['ScheduledDay', 'AppointmentDay'], axis=1)
features.head()

In [ ]:
model_data = features
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [ ]:
test_data.info()

Now we'll upload these files to S3.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('validation/validation.csv')).upload_file('validation.csv')

---
## Training and HPO

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
# container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='0.90-2')
container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='latest')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/train'.format(bucket), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/validation/'.format(bucket), content_type='csv')

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html).

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/output'.format(bucket),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        max_depth=7,
                        alpha=0.01,
                        eta=0.2,
                        gamma=4,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4,
                        min_child_weight=5,
                        subsample=0.8,
                        silent=0,
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## HPO


### Logarithmic scaling¶

Let us compare the results with executing a job using logarithmic scaling.


In [ ]:
# Setting HPO Config Files (log)
objective_metric_name = 'validation:auc'
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0.01, 10, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(0.01, 10, scaling_type="Logarithmic")
}

tuner_log = HyperparameterTuner(
    xgb,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=40,
    max_parallel_jobs=10,
    strategy='Bayesian'
)

tuner_log.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False, job_name="tuner-log" + time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime()))



### Linear scaling¶

Let us compare the results with executing a job using linear scaling.


In [ ]:
# Setting HPO Config Files (linear)
objective_metric_name = 'validation:auc'
hyperparameter_ranges_linear = {
    'alpha': ContinuousParameter(0.01, 10, scaling_type="Linear"),
    'lambda': ContinuousParameter(0.01, 10, scaling_type="Linear")
}
tuner_linear = HyperparameterTuner(
    xgb,
    objective_metric_name,
    hyperparameter_ranges_linear,
    max_jobs=40,
    max_parallel_jobs=10,
    strategy='Bayesian'
)

tuner_linear.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False, job_name="tuner-linear" + time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime()))
tuner_linear.wait()


In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_linear.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_log.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']


### Analyze tuning job results - after tuning job is completed

Once the tuning jobs have completed, we can compare the distribution of the hyperparameter configurations chosen in the two cases.

Please refer to "HPO_Analyze_TuningJob_Results.ipynb" to see more example code to analyze the tuning job results.


In [ ]:
# check jobs have finished
status_log = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_log.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']
status_linear = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_linear.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

assert status_log == 'Completed', "First must be completed, was {}".format(status_log)
assert status_linear == 'Completed', "Second must be completed, was {}".format(status_linear)

df_log = sagemaker.HyperparameterTuningJobAnalytics(tuner_log.latest_tuning_job.job_name).dataframe()
df_linear = sagemaker.HyperparameterTuningJobAnalytics(tuner_linear.latest_tuning_job.job_name).dataframe()
df_log['scaling'] = 'log'
df_linear['scaling'] = 'linear'
df = pd.concat([df_log, df_linear], ignore_index=True)

In [ ]:
g = sns.FacetGrid(df, col="scaling", palette='viridis')
g = g.map(plt.scatter, "alpha", "lambda", alpha=0.6)

---
## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
# Return name of the best training job for the latest "log" hyperparameter tuning job
best_training_job = tuner_log.best_training_job()
print(best_training_job)
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge',model_name=best_training_job, endpoint_name="XGBoost-best-HPO-model-{}".format(int(time.time())))

## Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = sagemaker.predictor.csv_serializer
xgb_predictor.deserializer = None

In [ ]:
# For CSV inference, the algorithm assumes that CSV input does not have the label column 
test_data.drop(labels='No_show', axis=1, inplace=True)

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
# Function to chunk down test set into smaller increments

def predict(data, model, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, model.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

## Generate predictions on the test set for the difference models

# predictions = xgb_predictor.predict predict(test_data.values, xgb_predictor)
predictions = predict(test_data.values, xgb_predictor)

There are many ways to compare the performance of a machine learning model, but let's start by simply by comparing actual to predicted values.  In this case, we're simply predicting whether the customer was a no-show (`1`) or not (`0`), which produces a simple confusion matrix.

In [ ]:
pd.crosstab(index=test_data.iloc[:, 1], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

An important point here is that because of the `np.round()` function above we are using a simple threshold (or cutoff) of 0.5.  Our predictions from `xgboost` come out as continuous values between 0 and 1 and we force them into the binary classes that we began with.  However, because a customer that is predicted to be in attendence but is in reality a "no-show" (FP) is expected to have the highest cost, we should consider adjusting this cutoff.  That will almost certainly increase the number of false negatives, but it can also be expected to increase the number of true positives and reduce the number of false negatives.

To get a rough intuition here, let's look at the continuous values of our predictions.

In [ ]:
plt.hist(predictions)
plt.show()

We can see that changing the cutoff from 0.5 to 0.3 results in 1 more true positives, 3 more false positives, and 1 fewer false negatives.  The numbers are small overall here, but that's 6-10% of customers overall that are shifting because of a change to the cutoff.  Was this the right decision?  We may end up retaining 3 extra customers, but we also unnecessarily incentivized 5 more customers who would have stayed.  Determining optimal cutoffs is a key step in properly applying machine learning in a real-world setting.  Let's discuss this more broadly and then apply a specific, hypothetical solution for our current problem.

### Relative cost of errors

Any practical binary classification problem is likely to produce a similarly sensitive cutoff. That by itself isn’t a problem. After all, if the scores for two classes are really easy to separate, the problem probably isn’t very hard to begin with and might even be solvable with simple rules instead of ML.

More important, if I put an ML model into production, there are costs associated with the model erroneously assigning false positives and false negatives. I also need to look at similar costs associated with correct predictions of true positives and true negatives.  Because the choice of the cutoff affects all four of these statistics, I need to consider the relative costs to the business for each of these four outcomes for each prediction.

#### Assigning costs


- Positive class -- "No-Show  " Patient 
- Negative class -- "Attending" Patient


  
  
What are the costs for our problem of "No-show" patients? The costs, of course, depend on the specific actions that the business takes. Let's make some assumptions here.

### TN

- Reality        : "Attending" Patient
- Model Predicts : "Attending" Patient
- Cost:\$0 

### FN

- Reality        : "Attending" Patient
- Model Predicts : "No-Show  " Patient 
- Cost:\$100

### FP

- Reality        : "No-Show  " Patient 
- Model Predicts : "Attending" Patient
- Cost:\$500

### TP

- Reality        : "No-Show  " Patient 
- Model Predicts : "No-Show  " Patient 
- Cost:\$0 

In [ ]:
cutoffs = (np.arange(0.01, 0.70, 0.01))

costs = []
for c in cutoffs:
    # TN / FN // FP / TP
    costs.append(np.sum(np.sum(np.array([[0, 100], [500, 0]]) * 
                               pd.crosstab(index=test_data.iloc[:, 1].values, columns=np.where(predictions > c, 1, 0)))))
                                        
costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of : $', np.min(costs))

The above chart shows how picking a threshold too low results in costs skyrocketing.  Meanwhile, setting the threshold too high results in the model classifying all customers as "No-shows", which ultimately grows to be nearly as costly. 

In [ ]:
pd.crosstab(index=test_data.iloc[:, 1], columns=np.where(predictions >= 0.01, 1, 0), rownames=['actual'], colnames=['predictions'])

## (Optional) Clean-up

If you're ready to be done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
# sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)